In [1]:
import pandas as pd
import gc
from google.colab import drive
drive.mount('/content/drive')

data_root = '/content/drive/MyDrive/부트캠프/'
data_path = data_root + '11_파이널/'

Mounted at /content/drive


In [2]:
# 딥러닝 모델 구조를 정의하는 것
from tensorflow.keras.models import Sequential
# 층구조를 정의하는 것
from tensorflow.keras.layers import Dense
# 활성화 함수를 정의하는 것
from tensorflow.keras.layers import Activation
# 다중 분류를 위한 원 핫 인코딩
from tensorflow.keras.utils import to_categorical
# 저장된 학습 모델 복원
from tensorflow.keras.models import load_model
# epoch마다 모델을 저장하는 함수
from tensorflow.keras.callbacks import ModelCheckpoint
# 조기 중단
from tensorflow.keras.callbacks import EarlyStopping
# 드롭아웃
from tensorflow.keras.layers import Dropout
# 배치노말
from tensorflow.keras.layers import BatchNormalization

# 딥러닝
import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
# gpu가 있다면..
if len(gpus) > 0 :
    try :
        for gpu in gpus :
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e :
        print(e)

In [3]:
df0 = pd.read_parquet(data_path + 'processed/train_12.parquet')

In [4]:
segment = pd.read_csv(data_path + 'processed/segment.csv')
segment.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2400000 entries, 0 to 2399999
Data columns (total 2 columns):
 #   Column   Dtype 
---  ------   ----- 
 0   ID       object
 1   Segment  object
dtypes: object(2)
memory usage: 36.6+ MB


In [5]:
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Columns: 312 entries, 기준년월 to 변동률_RV평잔
dtypes: float64(32), int16(75), int32(69), int8(135), object(1)
memory usage: 314.7+ MB


In [5]:
X = df0.drop(columns=['ID', '기준년월'])
y = segment.loc[2000000: ,'Segment']

In [6]:
from sklearn.preprocessing import RobustScaler

scaler = RobustScaler()
X_scaled = scaler.fit_transform(X)

In [8]:
# TO-DO Supervised Autoencoder

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=42)

In [8]:
from imblearn.combine import SMOTETomek
from imblearn.over_sampling import SMOTE

# sampler = SMOTETomek(sampling_strategy='not majority', random_state=42)
sampler = SMOTE(sampling_strategy='not majority', random_state=42)
# sampler = SMOTETomek(sampling_strategy='not majority', random_state=42)
# sampler = SMOTETomek(sampling_strategy='not majority', random_state=42)

In [9]:
train_x,train_y = smt.fit_resample(X_train,y_train)
train_y.value_counts()

,count
Segment,
B,240141
A,240141
C,239477
D,237677
E,237217


In [19]:
from tensorflow.keras import regularizers

In [73]:
# 모델 설정
model = Sequential()

model.add(Dense(256, input_dim=X_train.shape[1], kernel_regularizer=regularizers.L2(1e-5)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# model.add(Dense(256, kernel_regularizer=regularizers.l2(1e-3)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.25))

# model.add(Dense(240, kernel_regularizer=regularizers.L2(1e-3)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.25))

model.add(Dense(180, kernel_regularizer=regularizers.l2(1e-5)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(100, kernel_regularizer=regularizers.l2(1e-5)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

# model.add(Dense(64, kernel_regularizer=regularizers.l2(1e-4)))
# model.add(BatchNormalization())
# model.add(Activation('relu'))
# model.add(Dropout(0.25))

model.add(Dense(40, kernel_regularizer=regularizers.l2(1e-5)))
model.add(BatchNormalization())
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(5, activation='softmax'))

In [74]:
from tensorflow.keras.metrics import Precision, Recall
# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam',
              metrics=['accuracy',Precision(), Recall()])

In [13]:
# 자동 저장 콜백
call1 = ModelCheckpoint(filepath=data_path+'data/early_stop2.h5', monitor='val_loss', save_best_only=True)

In [68]:
# 조기 중단
# patience : 현재를 기준으로 몇회를 더 확인할 것인가.
# 현재의 오차가 지금까지의 오차보다 가장 낮은 수치를 보였다면 지금부터 patience 만큼
# 더 확인해 본다. 그 안에 오차가 더 작아지면 그 때부터 다시 patience 만큼을 확인한다.
# 만약 patience 만큼 더 살펴보았는데 더 이상 개선되지 않으면 학습이 중단된다.
call2 = EarlyStopping(monitor='val_loss', patience=25)

In [26]:
# 하이퍼 파라미터
epochs = 1000
batch_size = 1024

In [16]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_int = le.fit_transform(train_y)
train_y_one = to_categorical(y_int, num_classes=5)
test_y_one = to_categorical(le.transform(y_test), num_classes=5)

In [75]:
# 학습한다.
history = model.fit(train_x, train_y_one, epochs=epochs, batch_size=batch_size,
                      validation_data=(X_test, test_y_one), callbacks=[call1, call2])

Epoch 1/1000
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 18s 10ms/step - accuracy: 0.6604 - loss: 0.8047 - precision_12: 0.7672 - recall_12: 0.5339 - val_accuracy: 0.7355 - val_loss: 0.6500 - val_precision_12: 0.7845 - val_recall_12: 0.6892
Epoch 2/1000
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7904 - loss: 0.4962 - precision_12: 0.8286 - recall_12: 0.7405 - val_accuracy: 0.7218 - val_loss: 0.6678 - val_precision_12: 0.7762 - val_recall_12: 0.6667
Epoch 3/1000
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.7980 - loss: 0.4769 - precision_12: 0.8374 - recall_12: 0.7483 - val_accuracy: 0.7218 - val_loss: 0.6699 - val_precision_12: 0.7793 - val_recall_12: 0.6650
Epoch 4/1000
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8022 - loss: 0.4667 - precision_12: 0.8401 - recall_12: 0.7554 - val_accuracy: 0.7348 - val_loss: 0.6553 - val_precision_12: 0.7867 - val_recall_12: 0.6803
Epoch 5/1000
1167/1167 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - accuracy: 0.8073 - loss: 0.4579 - pr

In [76]:
a1 = model.evaluate(X_test, test_y_one)
display(f'loss : {a1[0]}')
display(f'accuracy : {a1[1]}')

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step - accuracy: 0.7526 - loss: 0.6333 - precision_12: 0.7955 - recall_12: 0.6984


'loss : 0.6324637532234192'

'accuracy : 0.7536200284957886'

In [25]:
r1 = model.predict(X_test)
r1

15000/15000 ━━━━━━━━━━━━━━━━━━━━ 18s 1ms/step


array([[3.3169621e-04, 6.3661988e-05, 5.9655689e-02, 1.7615952e-01,
        7.6378947e-01],
       [4.5088322e-10, 1.3127666e-09, 5.5187661e-06, 4.5129762e-04,
        9.9954319e-01],
       [3.3504268e-08, 3.7064908e-08, 1.0919254e-04, 3.3251673e-03,
        9.9656564e-01],
       ...,
       [0.0000000e+00, 1.1210388e-44, 2.6208566e-37, 4.8600355e-25,
        1.0000000e+00],
       [3.3169621e-04, 6.3661988e-05, 5.9655689e-02, 1.7615952e-01,
        7.6378947e-01],
       [0.0000000e+00, 2.1237519e-40, 1.6801917e-33, 1.7178755e-22,
        1.0000000e+00]], dtype=float32)

In [77]:
tf = pd.read_parquet(data_path + 'processed/test_12.parquet')

In [78]:
tf.drop(columns=['ID', '기준년월'], inplace=True)
tfx = scaler.transform(tf)

In [79]:
r1 = model.predict(tfx)

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 4s 1ms/step


In [30]:
display(y)

,Segment
0,D
1,E
2,C
3,D
4,E
...,...
2399995,E
2399996,D
2399997,C
2399998,E


In [80]:
import numpy as np

# 클래스 번호로 변환
y_pred_labels = np.argmax(r1, axis=1)

# 클래스 이름 리스트
class_names = ['A', 'B', 'C', 'D', 'E']

# 클래스 이름으로 매핑
y_pred_names = [class_names[i] for i in y_pred_labels]

In [43]:
display(y_pred_names)

['C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'B',
 'E',
 'C',
 'C',
 'B',
 'B',
 'E',
 'B',
 'E',
 'E',
 'B',
 'C',
 'B',
 'B',
 'C',
 'C',
 'C',
 'E',
 'C',
 'C',
 'E',
 'C',
 'E',
 'C',
 'C',
 'C',
 'C',
 'C',
 'B',
 'C',
 'E',
 'C',
 'E',
 'E',
 'E',
 'E',
 'C',
 'E',
 'C',
 'E',
 'C',
 'E',
 'C',
 'C',
 'C',
 'C',
 'C',
 'E',
 'C',
 'C',
 'C',
 'C',
 'C',
 'E',
 'C',
 'E',
 'C',
 'B',
 'D',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'E',
 'C',
 'C',
 'E',
 'C',
 'C',
 'B',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'D',
 'C',
 'E',
 'B',
 'C',
 'C',
 'E',
 'C',
 'E',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'E',
 'C',
 'C',
 'B',
 'E',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'E',
 'C',
 'E',
 'C',
 'C',
 'C',
 'B',
 'E',
 'C',
 'B',
 'C',
 'C',
 'C',
 'C',
 'C',
 'C',
 'B',
 'C',
 'E',
 'E',
 'E',
 'C',
 'E',
 'C',
 'E',
 'C',
 'B',
 'B',
 'C',
 'E',
 'E',
 'C',
 'C',
 'E',
 'C',
 'E',
 'C',
 'C',
 'C',
 'C',
 'E',
 'C',
 'B',
 'C',
 'C',
 'E',
 'C'

In [81]:
sub = pd.read_csv(data_path + 'sample_submission.csv')
sub

,ID,Segment
0,TEST_00000,A
1,TEST_00001,A
2,TEST_00002,A
3,TEST_00003,A
4,TEST_00004,A
...,...,...
99995,TEST_99995,A
99996,TEST_99996,A
99997,TEST_99997,A
99998,TEST_99998,A


In [82]:
len(y_pred_names)

100000

In [83]:
sub['Segment']= y_pred_names

In [84]:
sub['Segment'].value_counts()

,count
Segment,
E,67487
D,24411
C,8054
A,44
B,4


In [85]:
sub.to_csv(data_path + 'submission2.csv')